In [1]:
#Importación de librerias útiles
import pandas
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')

#Librerias cientificas scipy, numpy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para aplicar Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')

# Calculo de la mediana

In [2]:
text_list=corpus_filtro4['text'][:]
category_list=corpus_filtro4['category'][:]
h1_list=corpus_filtro4['1'][:]
h2_list=corpus_filtro4['2'][:]
h3_list=corpus_filtro4['3'][:]
h4_list=corpus_filtro4['4'][:]
h5_list=corpus_filtro4['5'][:]
medians=[]
pos=0
for i in range( len(corpus_filtro4)):
    values = []
    for j in range( h1_list[i]):
        values.append(1)
    for j in range( h2_list[i] ):
        values.append(2)
    for j in range( h3_list[i] ):
        values.append(3)
    for j in range( h4_list[i] ):
        values.append(4)
    for j in range( h5_list[i] ):
        values.append(5)
    if len(values) == 0:
        values.append(0)  
    mediana=np.median(values)
    if( len(values) % 2 == 1 ):
        medians.append(math.floor(mediana))
    else:
        for i in values:
            if( i >= mediana):
                medians.append(i)
                break
    pos+=1
        
print(len(medians), " ", len(corpus_filtro4))

3438   3438


In [3]:
print("Agregando informacion de mediana")
d = {'text' : corpus_filtro4['text'][:],
    'category' : corpus_filtro4['category'][:],
    'median': medians,
    '1':corpus_filtro4['1'][:],
    '2':corpus_filtro4['2'][:],
    '3':corpus_filtro4['3'][:],
    '4':corpus_filtro4['4'][:],
    '5':corpus_filtro4['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', 'median', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro4median.csv')
print("Tweets guardados en corpus_filtro4median.csv")

Agregando informacion de mediana
Tweets guardados en corpus_filtro4median.csv


# Clasificador 1 mediana con valor 1 o no

In [5]:
vectorizer_c1 = CountVectorizer(stop_words='english')

In [6]:
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
text_list_c1=corpus_filtro4median['text'][:]
median_list_c1=corpus_filtro4median['median'][:]
category_list_c1=[]
for i in median_list_c1:
    if(i == 1):
        category_list_c1.append(1)
    else:
        category_list_c1.append(0)

pos=0
for t in text_list_c1:
    if type(t) != str:
        text_list_c1[pos]="NaN"
    pos+=1
        
train_features_c1 = vectorizer.fit_transform(text_list_c1)

In [7]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
text_devlist_c1=corpus_filtro4median['text']
#category_devlist=corpus_filtro5_devset['category'][:]
pos=0
for t in text_devlist_c1:
    if type(t) != str:
        text_devlist_c1[pos]="NaN"
    pos+=1
test_features_c1 = vectorizer.transform(text_devlist_c1)


In [8]:
nb_c1 = MultinomialNB()
nb_c1.fit(train_features_c1 , category_list_c1[:])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
from sklearn.naive_bayes import MultinomialNB
nb_c1.predict_proba(test_features_c1)

array([[  9.96691962e-01,   3.30803785e-03],
       [  9.99986382e-01,   1.36177079e-05],
       [  9.99861132e-01,   1.38868497e-04],
       ..., 
       [  9.99968796e-01,   3.12036008e-05],
       [  9.93729189e-01,   6.27081062e-03],
       [  9.99996416e-01,   3.58406828e-06]])

In [10]:
nb_c1.predict(test_features_c1)

array([0, 0, 0, ..., 0, 0, 0])

# Clasificador 2 mediana con valor 2 o no

In [11]:
vectorizer_c2 = CountVectorizer(stop_words='english')

In [12]:
#corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
#text_list_c1=corpus_filtro4median['text'][:]
#median_list_c1=corpus_filtro4median['median'][:]
category_list_c2=[]
for i in median_list_c1:
    if(i == 2):
        category_list_c2.append(1)
    else:
        category_list_c2.append(0)

#train_features_c2 = vectorizer.fit_transform(text_list_c1[:100])

In [27]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
#text_devlist_c1=corpus_filtro4median['text'][600:605]
#category_devlist=corpus_filtro5_devset['category'][:]
#test_features_c1 = vectorizer.transform(text_devlist_c1)


In [13]:
nb_c2 = MultinomialNB()
nb_c2.fit(train_features_c1 , category_list_c2[:])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
from sklearn.naive_bayes import MultinomialNB
nb_c2.predict_proba(test_features_c1)

array([[  2.50621004e-02,   9.74937900e-01],
       [  9.99992432e-01,   7.56793676e-06],
       [  9.99792510e-01,   2.07489892e-04],
       ..., 
       [  9.99653143e-01,   3.46857060e-04],
       [  8.84755689e-01,   1.15244311e-01],
       [  9.21415815e-01,   7.85841853e-02]])

In [15]:
nb_c2.predict(test_features_c1)

array([1, 0, 0, ..., 0, 0, 0])

# Clasificador 3 mediana con valor 3 o no

In [16]:
vectorizer_c3 = CountVectorizer(stop_words='english')

In [17]:
#corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
#text_list_c1=corpus_filtro4median['text'][:]
#median_list_c1=corpus_filtro4median['median'][:]
category_list_c3=[]
for i in median_list_c1:
    if(i == 3):
        category_list_c3.append(1)
    else:
        category_list_c3.append(0)

#train_features_c2 = vectorizer.fit_transform(text_list_c1[:100])

In [27]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
#text_devlist_c1=corpus_filtro4median['text'][600:605]
#category_devlist=corpus_filtro5_devset['category'][:]
#test_features_c1 = vectorizer.transform(text_devlist_c1)


In [18]:
nb_c3 = MultinomialNB()
nb_c3.fit(train_features_c1 , category_list_c3[:])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
from sklearn.naive_bayes import MultinomialNB
nb_c3.predict_proba(test_features_c1)

array([[ 0.98398656,  0.01601344],
       [ 0.9982972 ,  0.0017028 ],
       [ 0.0428097 ,  0.9571903 ],
       ..., 
       [ 0.94713124,  0.05286876],
       [ 0.98673773,  0.01326227],
       [ 0.97049918,  0.02950082]])

In [20]:
nb_c3.predict(test_features_c1)

array([0, 0, 1, ..., 0, 0, 0])

# Clasificador 4 mediana con valor 4 o no

In [21]:
vectorizer_c4 = CountVectorizer(stop_words='english')

In [22]:
#corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
#text_list_c1=corpus_filtro4median['text'][:]
#median_list_c1=corpus_filtro4median['median'][:]
category_list_c4=[]
for i in median_list_c1:
    if(i == 4):
        category_list_c4.append(1)
    else:
        category_list_c4.append(0)

#train_features_c2 = vectorizer.fit_transform(text_list_c1[:100])

In [43]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
#text_devlist_c1=corpus_filtro4median['text'][600:605]
#category_devlist=corpus_filtro5_devset['category'][:]
#test_features_c1 = vectorizer.transform(text_devlist_c1)


In [23]:
nb_c4 = MultinomialNB()
nb_c4.fit(train_features_c1 , category_list_c4[:])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
from sklearn.naive_bayes import MultinomialNB
nb_c4.predict_proba(test_features_c1)

array([[  9.52274184e-01,   4.77258161e-02],
       [  9.99813166e-01,   1.86834336e-04],
       [  9.81911464e-01,   1.80885361e-02],
       ..., 
       [  9.98076840e-01,   1.92316036e-03],
       [  9.44985036e-01,   5.50149635e-02],
       [  9.41407671e-01,   5.85923285e-02]])

In [25]:
nb_c4.predict(test_features_c1)

array([0, 0, 0, ..., 0, 0, 0])

# Clasificador 5 mediana con valor 5 o no

In [26]:
vectorizer_c5 = CountVectorizer(stop_words='english')

In [27]:
#corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
#text_list_c1=corpus_filtro4median['text'][:]
#median_list_c1=corpus_filtro4median['median'][:]
category_list_c5=[]
for i in median_list_c1:
    if(i == 5):
        category_list_c5.append(1)
    else:
        category_list_c5.append(0)

#train_features_c2 = vectorizer.fit_transform(text_list_c1[:100])

In [49]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
#text_devlist_c1=corpus_filtro4median['text'][600:605]
#category_devlist=corpus_filtro5_devset['category'][:]
#test_features_c1 = vectorizer.transform(text_devlist_c1)


In [29]:
nb_c5 = MultinomialNB()
nb_c5.fit(train_features_c1 , category_list_c5)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
from sklearn.naive_bayes import MultinomialNB
nb_c5.predict_proba(test_features_c1)

array([[  9.99966226e-01,   3.37740971e-05],
       [  9.99999994e-01,   6.13796472e-09],
       [  9.99569441e-01,   4.30559172e-04],
       ..., 
       [  9.99935561e-01,   6.44394245e-05],
       [  9.99828758e-01,   1.71242216e-04],
       [  9.99999764e-01,   2.35666968e-07]])

In [31]:
nb_c5.predict(test_features_c1)

array([0, 0, 0, ..., 0, 0, 0])

# Clasificador 6 mediana con valor 0 o no

In [32]:
vectorizer_c0 = CountVectorizer(stop_words='english')

In [33]:
#corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
#text_list_c1=corpus_filtro4median['text'][:]
#median_list_c1=corpus_filtro4median['median'][:]
category_list_c0=[]
for i in median_list_c1:
    if(i == 0):
        category_list_c0.append(1)
    else:
        category_list_c0.append(0)

#train_features_c2 = vectorizer.fit_transform(text_list_c1[:100])

In [55]:
#corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
#text_devlist_c1=corpus_filtro4median['text'][600:605]
#category_devlist=corpus_filtro5_devset['category'][:]
#test_features_c1 = vectorizer.transform(text_devlist_c1)


In [34]:
nb_c0 = MultinomialNB()
nb_c0.fit(train_features_c1 , category_list_c0[:])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
from sklearn.naive_bayes import MultinomialNB
nb_c0.predict_proba(test_features_c1)

array([[  9.99997893e-01,   2.10690425e-06],
       [  9.99999275e-01,   7.24650982e-07],
       [  9.99999695e-01,   3.04533822e-07],
       ..., 
       [  8.43031705e-01,   1.56968295e-01],
       [  9.99966854e-01,   3.31456083e-05],
       [  9.99999948e-01,   5.19934896e-08]])

In [36]:
nb_c0.predict(test_features_c1)

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
c1_pred=nb_c1.predict_proba(test_features_c1)
c2_pred=nb_c2.predict_proba(test_features_c1)
c3_pred=nb_c3.predict_proba(test_features_c1)
c4_pred=nb_c4.predict_proba(test_features_c1)
c5_pred=nb_c5.predict_proba(test_features_c1)
c0_pred=nb_c0.predict_proba(test_features_c1)

In [40]:
predicts=[]
proba=[]
for i in range( len(c1_pred) ):
    maximum=np.max([c1_pred[i][1],c2_pred[i][1],c3_pred[i][1],c4_pred[i][1],c5_pred[i][1],c0_pred[i][1]])
    if(maximum == c1_pred[i][1]):
        predicts.append(1)
    elif(maximum == c2_pred[i][1]):
        predicts.append(2)
    elif(maximum == c3_pred[i][1]):
        predicts.append(3)
    elif(maximum == c4_pred[i][1]):
        predicts.append(4)
    elif(maximum == c5_pred[i][1]):
        predicts.append(5)
    elif(maximum == c0_pred[i][1]):
        predicts.append(0)
    proba.append(maximum)

In [41]:
pos=0
for pred in predicts:
    print("Posicion", pos, "Mediana  ", pred, " probabilidad ", proba[pos])
    pos+=1

Posicion 0 Mediana  2  probabilidad  0.974937899608
Posicion 1 Mediana  3  probabilidad  0.0017027998876
Posicion 2 Mediana  3  probabilidad  0.957190303814
Posicion 3 Mediana  2  probabilidad  0.466014830496
Posicion 4 Mediana  3  probabilidad  0.110604755838
Posicion 5 Mediana  4  probabilidad  0.0758266150931
Posicion 6 Mediana  1  probabilidad  0.0127856367296
Posicion 7 Mediana  4  probabilidad  0.0884796712328
Posicion 8 Mediana  4  probabilidad  0.0495036495831
Posicion 9 Mediana  2  probabilidad  0.965352102377
Posicion 10 Mediana  4  probabilidad  0.351591950716
Posicion 11 Mediana  0  probabilidad  0.150576557514
Posicion 12 Mediana  3  probabilidad  0.019651662632
Posicion 13 Mediana  3  probabilidad  0.596256848221
Posicion 14 Mediana  4  probabilidad  0.0368899483541
Posicion 15 Mediana  0  probabilidad  0.99194520056
Posicion 16 Mediana  4  probabilidad  0.285095089724
Posicion 17 Mediana  3  probabilidad  0.56310206015
Posicion 18 Mediana  0  probabilidad  0.999864176759

Posicion 171 Mediana  3  probabilidad  0.57360590084
Posicion 172 Mediana  3  probabilidad  0.999810117785
Posicion 173 Mediana  3  probabilidad  0.832775764703
Posicion 174 Mediana  4  probabilidad  0.212480873015
Posicion 175 Mediana  0  probabilidad  0.999788444058
Posicion 176 Mediana  2  probabilidad  0.25678491498
Posicion 177 Mediana  4  probabilidad  0.0240239004623
Posicion 178 Mediana  3  probabilidad  0.997418350276
Posicion 179 Mediana  4  probabilidad  0.00137605220417
Posicion 180 Mediana  0  probabilidad  0.256231777624
Posicion 181 Mediana  4  probabilidad  0.0460848927029
Posicion 182 Mediana  3  probabilidad  0.142815300658
Posicion 183 Mediana  5  probabilidad  0.836671454992
Posicion 184 Mediana  3  probabilidad  0.0314027126185
Posicion 185 Mediana  3  probabilidad  0.0351463276452
Posicion 186 Mediana  2  probabilidad  0.0105872343263
Posicion 187 Mediana  2  probabilidad  0.990237752986
Posicion 188 Mediana  3  probabilidad  0.0336397490622
Posicion 189 Mediana  

Posicion 362 Mediana  0  probabilidad  0.991852862006
Posicion 363 Mediana  4  probabilidad  0.987642377766
Posicion 364 Mediana  3  probabilidad  0.000303389736175
Posicion 365 Mediana  3  probabilidad  0.0247440930378
Posicion 366 Mediana  0  probabilidad  0.711544074409
Posicion 367 Mediana  3  probabilidad  0.175684396917
Posicion 368 Mediana  0  probabilidad  0.995796916605
Posicion 369 Mediana  4  probabilidad  0.225671195454
Posicion 370 Mediana  0  probabilidad  0.781441566972
Posicion 371 Mediana  4  probabilidad  0.0680088500355
Posicion 372 Mediana  4  probabilidad  0.717755264767
Posicion 373 Mediana  4  probabilidad  0.0881762915942
Posicion 374 Mediana  1  probabilidad  0.947332437259
Posicion 375 Mediana  1  probabilidad  0.988762435933
Posicion 376 Mediana  3  probabilidad  0.924486849275
Posicion 377 Mediana  3  probabilidad  0.0462486940086
Posicion 378 Mediana  4  probabilidad  0.00107738468174
Posicion 379 Mediana  0  probabilidad  0.999848200929
Posicion 380 Median

Posicion 538 Mediana  0  probabilidad  0.728598224928
Posicion 539 Mediana  4  probabilidad  0.217336039889
Posicion 540 Mediana  3  probabilidad  0.0172254364502
Posicion 541 Mediana  3  probabilidad  0.922777066614
Posicion 542 Mediana  0  probabilidad  0.20124807401
Posicion 543 Mediana  0  probabilidad  0.124383071908
Posicion 544 Mediana  0  probabilidad  0.80833051629
Posicion 545 Mediana  3  probabilidad  0.247998193262
Posicion 546 Mediana  0  probabilidad  0.187159079867
Posicion 547 Mediana  3  probabilidad  0.639887619684
Posicion 548 Mediana  0  probabilidad  0.998471504716
Posicion 549 Mediana  4  probabilidad  0.445072850945
Posicion 550 Mediana  4  probabilidad  0.679638844405
Posicion 551 Mediana  0  probabilidad  0.916934832872
Posicion 552 Mediana  0  probabilidad  0.999026450056
Posicion 553 Mediana  4  probabilidad  0.00199281136151
Posicion 554 Mediana  1  probabilidad  0.0535067252884
Posicion 555 Mediana  0  probabilidad  0.513524508169
Posicion 556 Mediana  0  p

Posicion 733 Mediana  3  probabilidad  0.16158766669
Posicion 734 Mediana  0  probabilidad  0.0154998575722
Posicion 735 Mediana  3  probabilidad  0.763828344129
Posicion 736 Mediana  0  probabilidad  0.0167344417865
Posicion 737 Mediana  4  probabilidad  0.129343078348
Posicion 738 Mediana  0  probabilidad  0.998490196117
Posicion 739 Mediana  3  probabilidad  0.00570747957985
Posicion 740 Mediana  2  probabilidad  0.72769313215
Posicion 741 Mediana  0  probabilidad  0.93948657828
Posicion 742 Mediana  3  probabilidad  0.371381806125
Posicion 743 Mediana  4  probabilidad  0.351145642886
Posicion 744 Mediana  4  probabilidad  0.297958077671
Posicion 745 Mediana  3  probabilidad  0.667810362568
Posicion 746 Mediana  1  probabilidad  0.00558052970605
Posicion 747 Mediana  3  probabilidad  0.047766517307
Posicion 748 Mediana  3  probabilidad  0.0154050003025
Posicion 749 Mediana  4  probabilidad  0.778574927027
Posicion 750 Mediana  4  probabilidad  0.34924020106
Posicion 751 Mediana  0  

...

Posicion 2605 Mediana  3  probabilidad  0.617799761409
Posicion 2606 Mediana  4  probabilidad  0.915609755439
Posicion 2607 Mediana  0  probabilidad  0.0508452377638
Posicion 2608 Mediana  4  probabilidad  0.616976029846
Posicion 2609 Mediana  0  probabilidad  0.0586980467139
Posicion 2610 Mediana  4  probabilidad  0.0634182070937
Posicion 2611 Mediana  0  probabilidad  0.694636226388
Posicion 2612 Mediana  4  probabilidad  0.206411551894
Posicion 2613 Mediana  4  probabilidad  0.0126563561905
Posicion 2614 Mediana  0  probabilidad  0.999972133743
Posicion 2615 Mediana  4  probabilidad  0.0417127301938
Posicion 2616 Mediana  2  probabilidad  0.995321779374
Posicion 2617 Mediana  0  probabilidad  0.11686229565
Posicion 2618 Mediana  3  probabilidad  0.128261173465
Posicion 2619 Mediana  3  probabilidad  0.230227574528
Posicion 2620 Mediana  4  probabilidad  0.00697481356741
Posicion 2621 Mediana  3  probabilidad  0.585069130189
Posicion 2622 Mediana  3  probabilidad  0.999605462804
Posi

Posicion 2777 Mediana  4  probabilidad  0.636965278336
Posicion 2778 Mediana  0  probabilidad  0.998810089988
Posicion 2779 Mediana  0  probabilidad  0.997137193462
Posicion 2780 Mediana  4  probabilidad  0.0434568338212
Posicion 2781 Mediana  3  probabilidad  0.0811526232435
Posicion 2782 Mediana  3  probabilidad  0.0853521557868
Posicion 2783 Mediana  0  probabilidad  0.148636352035
Posicion 2784 Mediana  4  probabilidad  0.00321569742528
Posicion 2785 Mediana  0  probabilidad  0.0329654234853
Posicion 2786 Mediana  3  probabilidad  0.0189246675383
Posicion 2787 Mediana  2  probabilidad  0.719178639364
Posicion 2788 Mediana  4  probabilidad  0.0727107100504
Posicion 2789 Mediana  0  probabilidad  0.812606987557
Posicion 2790 Mediana  4  probabilidad  0.671241976319
Posicion 2791 Mediana  3  probabilidad  0.355763733326
Posicion 2792 Mediana  0  probabilidad  0.069157219806
Posicion 2793 Mediana  4  probabilidad  0.027478800299
Posicion 2794 Mediana  1  probabilidad  0.0155124193893
P

Posicion 2958 Mediana  0  probabilidad  0.0735203700721
Posicion 2959 Mediana  4  probabilidad  0.813501287158
Posicion 2960 Mediana  4  probabilidad  0.105462148014
Posicion 2961 Mediana  0  probabilidad  0.00175898598308
Posicion 2962 Mediana  4  probabilidad  0.0209064670449
Posicion 2963 Mediana  3  probabilidad  0.877772741613
Posicion 2964 Mediana  0  probabilidad  0.0216878933298
Posicion 2965 Mediana  3  probabilidad  0.0282841407612
Posicion 2966 Mediana  3  probabilidad  0.193778100013
Posicion 2967 Mediana  3  probabilidad  0.974638332477
Posicion 2968 Mediana  0  probabilidad  0.998415241575
Posicion 2969 Mediana  0  probabilidad  0.999999313751
Posicion 2970 Mediana  2  probabilidad  0.225915338298
Posicion 2971 Mediana  2  probabilidad  0.0226120607838
Posicion 2972 Mediana  0  probabilidad  0.62944756745
Posicion 2973 Mediana  3  probabilidad  0.0947143380688
Posicion 2974 Mediana  1  probabilidad  0.755901313026
Posicion 2975 Mediana  3  probabilidad  0.00439393344446
P

Posicion 3127 Mediana  3  probabilidad  0.0059148031341
Posicion 3128 Mediana  0  probabilidad  0.604554662266
Posicion 3129 Mediana  3  probabilidad  0.0509698464767
Posicion 3130 Mediana  3  probabilidad  0.074987573103
Posicion 3131 Mediana  4  probabilidad  0.320406473116
Posicion 3132 Mediana  2  probabilidad  0.858393178413
Posicion 3133 Mediana  0  probabilidad  0.202699365987
Posicion 3134 Mediana  0  probabilidad  0.76819577872
Posicion 3135 Mediana  3  probabilidad  0.022952624766
Posicion 3136 Mediana  2  probabilidad  0.999693041073
Posicion 3137 Mediana  3  probabilidad  0.999999999803
Posicion 3138 Mediana  3  probabilidad  0.143806859157
Posicion 3139 Mediana  3  probabilidad  0.14880473401
Posicion 3140 Mediana  3  probabilidad  0.0505322524954
Posicion 3141 Mediana  3  probabilidad  0.00522682901363
Posicion 3142 Mediana  3  probabilidad  0.155693133073
Posicion 3143 Mediana  4  probabilidad  0.0083383519951
Posicion 3144 Mediana  3  probabilidad  0.261684453193
Posici

Posicion 3330 Mediana  1  probabilidad  0.0132546483599
Posicion 3331 Mediana  3  probabilidad  0.249962235911
Posicion 3332 Mediana  1  probabilidad  0.831899700221
Posicion 3333 Mediana  0  probabilidad  0.734991618208
Posicion 3334 Mediana  3  probabilidad  0.562309381451
Posicion 3335 Mediana  3  probabilidad  0.000186068288303
Posicion 3336 Mediana  3  probabilidad  0.248400232693
Posicion 3337 Mediana  0  probabilidad  0.872049197907
Posicion 3338 Mediana  3  probabilidad  0.991625290911
Posicion 3339 Mediana  4  probabilidad  0.051361546114
Posicion 3340 Mediana  1  probabilidad  0.999995557937
Posicion 3341 Mediana  2  probabilidad  0.714108540408
Posicion 3342 Mediana  0  probabilidad  0.951352491253
Posicion 3343 Mediana  0  probabilidad  0.999500810795
Posicion 3344 Mediana  2  probabilidad  0.45457485214
Posicion 3345 Mediana  3  probabilidad  0.0919806908468
Posicion 3346 Mediana  4  probabilidad  0.908523500951
Posicion 3347 Mediana  0  probabilidad  0.0557391915147
Posic